In [169]:
%reset 


In [170]:
#import utilities 
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from chromedriver_py import binary_path   
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.action_chains import ActionChains
from openpyxl import Workbook
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import io


In [171]:
# Function to initialize Chrome WebDriver
def initialize_chrome():
    chrome_options = ChromeOptions()
    chrome_options.add_argument('log-level=3')
    chrome_options.add_argument('--ignore-certificate-errors-spki-list')
    chrome_options.add_argument('--ignore-ssl-errors')
    #chrome_options.add_argument('--headless')  # You can uncomment this if you want to run Chrome in headless mode
    chrome_options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(service=Service(), options=chrome_options)
    driver.maximize_window()  # Maximize the window
    return driver
# Function to initialize Firefox WebDriver
def initialize_firefox():
    firefox_options = FirefoxOptions()
    firefox_options.headless = True  # Run Firefox in headless mode
    driver = webdriver.Firefox(options=firefox_options)
    driver.maximize_window()  # Maximize the window
    return driver
# Function to initialize WebDriver
def initialize_driver():
    try:
        driver = initialize_chrome()
        print("Chrome WebDriver initialized successfully.")
        return driver
    except Exception as e:
        print("Chrome WebDriver initialization failed:", e)
        print("Switching to chrome WebDriver...")
        return initialize_firefox()

# Function to wait for element visibility
def wait_for_element_visibility(driver, by, selector, timeout=20):
    return WebDriverWait(driver, timeout).until(
        EC.visibility_of_element_located((by, selector))
    )



In [172]:
driver = initialize_driver()

driver.get("https://dispute.delhigovt.nic.in/frm_searchdisputedproperty.aspx?var=wNXACD7M+tg=&Digest=HRB2a7xKMHtq3BA3FqvhhQ")



Chrome WebDriver initialized successfully.


In [173]:


def scrape_page(driver): 
    # Wait for the table to be visible
    table = wait_for_element_visibility(driver, By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_grd_complaintdetail"]')

    table_html = table.get_attribute('outerHTML')
    # Convert the HTML table to a DataFrame
    df = pd.read_html(io.StringIO(table_html))[0]

    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df["District"] = "Central" 
    df["SRO_NAME"] = "Sub Registrar I-Kashmere Gate" 
    # Remove the last two rows
    df = df.iloc[:-2]
    return df



In [174]:
import math 

def scrape_all_pages(driver, num_pages, scrape_table):
    final_data = scrape_table(driver)
    max_attempts = math.ceil(num_pages/10)  # Calculate the maximum number of attempts needed to scrape all pages
    
    total_checks = [i for i in range(1, max_attempts + 1)]  # Create a list of total checks
    
    # Loop through each attempt
    for i in range(1, max_attempts + 1, 1): 
        # Check if it's the first or last attempt
        if i ==1 : 
            # Loop through each page element (from 2 to 11)
            for j in range(2, 12, 1):
                # Scroll to the bottom of the page
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(3)
                # Construct the XPath to find the next page element
                next_element_path = f'//*[@id="ctl00_ContentPlaceHolder1_grd_complaintdetail"]/tbody/tr[17]/td/table/tbody/tr/td[{j}]/a'
                # Find the next page element by its XPath
                next_page_element = WebDriverWait(driver, 10).until(
                   EC.presence_of_element_located((By.XPATH, next_element_path))
                )
                # Click on the next page element
                actions = ActionChains(driver)
                actions.click(next_page_element).perform()
                # Wait for the table to appear
                wait_for_element_visibility(driver, By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_grd_complaintdetail"]')
                # Scrape data from the current page
                temp_df = scrape_table(driver)
                # Concatenate the scraped data with the final data
                final_data = pd.concat([final_data, temp_df])
        elif  total_checks[-1] == i: 

               for m in range(4, 12, 1):
                # Scroll to the bottom of the page
                  driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                  time.sleep(3)
                # Construct the XPath to find the next page element
                  next_element_path = f'//*[@id="ctl00_ContentPlaceHolder1_grd_complaintdetail"]/tbody/tr[17]/td/table/tbody/tr/td[{m}]/a'
                # Find the next page element by its XPath
                  next_page_element = WebDriverWait(driver, 10).until(
                   EC.presence_of_element_located((By.XPATH, next_element_path))
                  )
                # Click on the next page element
                  actions = ActionChains(driver)
                  actions.click(next_page_element).perform()
                # Wait for the table to appear
                  wait_for_element_visibility(driver, By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_grd_complaintdetail"]')
                # Scrape data from the current page
                  temp_df = scrape_table(driver)
                 # Concatenate the scraped data with the final data
                  final_data = pd.concat([final_data, temp_df])

        else:
            # Loop through each page element (from 3 to 12)
            for k in range(3, 13, 1): 
                # Scroll to the bottom of the page
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                time.sleep(3)
               
                next_element_path = f'//*[@id="ctl00_ContentPlaceHolder1_grd_complaintdetail"]/tbody/tr[17]/td/table/tbody/tr/td[{k}]/a'

                # Find the "..." element by its text
                next_page_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, next_element_path))
                )
                # Click on the "..." element using JavaScript
                driver.execute_script("arguments[0].click();", next_page_element)
                # Wait for the table to appear
                wait_for_element_visibility(driver, By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_grd_complaintdetail"]')
                # Scrape data from the current page
                temp_df = scrape_table(driver)
                # Concatenate the scraped data with the final data
                final_data = pd.concat([final_data, temp_df])
            time.sleep(5)
        print(f"Successfully scraped for full page {i}")

    return final_data


In [175]:
num_pages = 29  # Number of pages to scrape

# Call the scrape_all_pages function
all_data = scrape_all_pages(driver, num_pages, scrape_table= scrape_page)

# Close the WebDriver
driver.quit()  # No issues with closing the WebDriver

Successfully scraped for full page 1
Successfully scraped for full page 2
Successfully scraped for full page 3


In [176]:
num_pages =29
max_attempts = math.ceil(num_pages/10)  # Calculate the maximum number of attempts needed to scrape all pages
total_checks = [i for i in range(1, max_attempts + 1)]  # Create a list of total checks
print(max_attempts, total_checks)

3 [1, 2, 3]


In [177]:
all_data.to_excel("C:\\Users\\niranjan.kumar\\Desktop\\Property_Tax\\Raw_Urban_Data\\Delhi_Dispute\\CENTRAL_DELHI.xlsx", index= False)